Объявляем системные переменные

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
import os
import glob
import re
import numpy as np
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

path_to_models = '/kaggle/input/modelsnapshots'

# path_to_witcher_1 = '/kaggle/input/witcher-book'
# path_to_lotr_2 = '/kaggle/input/lord-of-the-ring'
# path_to_got_3 = '/kaggle/input/game-of-thrones-book-files'
path_to_all = '/kaggle/input/fantasy-books'

output_prefix = 'fantasy'

препроцессинг текстов и функция, которая возвращает последний снепшот модели из папки снепшотов

In [ ]:
def prep(path_to_data, list_of_true_names, max_len_of_sent, max_len_of_paragraph):
    list_of_files = [os.path.join(path_to_data, i) for i in list_of_true_names]
#     print(list_of_files)
    text = None
    for file in list_of_files:
        with open(file, 'r') as f:
            if text is None:
                text = f.read()
            else:
                text += f.read()
    text = text.replace('\n', ' ')
    text = text.replace('***', '')
    text = text.replace('“', "'")
    text = text.replace('”', "'")
    text = text.replace('’', "'")
    text = text.replace('‘', "'")
    text = sent_tokenize(text)
    print("top 20 the longest sent: " + str(sorted([len(sent.split()) for sent in text], reverse=True)[:20]))
    print(f"Deleteing sentences with more than {max_len_of_sent} words...")
    list_of_lens = [len(sent.split()) for sent in text if len(sent.split()) < max_len_of_sent]
    text = [sent for sent in text if len(sent.split()) < max_len_of_sent]
#     return text

    print(f"Perform paragraphs with less than {max_len_of_paragraph} words from sentences...")
    list_of_paragraph_lens = []
    improved_text_array = []
    i = 0
    list_of_paragraph_lens.append([list_of_lens[0]])
    improved_text_array.append(text[0])
    for j in range(1, len(list_of_lens)-2):
        if sum(list_of_paragraph_lens[i]) + list_of_lens[j] < max_len_of_paragraph:
            list_of_paragraph_lens[i].append(list_of_lens[j])
            improved_text_array[i] += ' ' + text[j]
        else:
            list_of_paragraph_lens.append([list_of_lens[j]])
            improved_text_array.append(text[j])
            i += 1
    improved_text_array[i] += ' ' + text[-1]
#     return improved_text_array
          
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    list_of_tokens = [len(tokenizer.encode(sent)) for sent in improved_text_array]
    print("top 20 the longest tokens: " + str(sorted(list_of_tokens, reverse=True)[:20]))
    print("top 20 the smallest tokens: " + str(sorted(list_of_tokens)[:20]))
    sns.histplot(list_of_tokens)
    return improved_text_array, list_of_tokens

In [ ]:
witcher_order = [
        'Andrzej_Sapkowski_-_The_Last_Wish.txt',
        '5_Sword_of_Destiny_The_Witcher_5_by_Andrzej_Sapkowski.txt',
        '2_Blood_of_Elves_The_Witcher_Book_2_by_Sapkowski_Andrzej.txt',
        '3_The_Time_of_Contempt_The_Witcher_Book_3_by_Sapkowski_Andrzej.txt',
        '4_Baptism_of_Fire_The_Witcher_4_by_Andrzej_Sapkowski.txt',
        '6_The_Tower_of_the_Swallow_The_Witcher_6_by_Andrzej_Sapkowski.txt',
        'Andrzej_Sapkowski_-_The_Lady_of_the_Lake.txt',
        'Andrzej_Sapkowski_-_Season_of_Storms.txt'
]

got_order = ['got5.txt', 'got1.txt', 'got2.txt', 'got4.txt', 'got3.txt']

lotr_order = [
        'dzhon_ronald_ruel_tolkin-the_hobbit.txt',
        'dzhon_ronald_ruel_tolkin-the_lord_of_the_rings.txt'
]

In [ ]:
# text_got, list_of_tokens_got = prep(path_to_all, got_order, max_len_of_sent=200, max_len_of_paragraph=500)
# print(len(text_got))
# text_got[np.argmin(np.array(list_of_tokens_got))], text_got[np.argmax(np.array(list_of_tokens_got))]

In [ ]:
# text_witcher, list_of_tokens_witcher = prep(path_to_all, witcher_order, max_len_of_sent=200, max_len_of_paragraph=500)
# print(len(text_witcher))
# text_witcher[np.argmin(np.array(list_of_tokens_witcher))], text_witcher[np.argmax(np.array(list_of_tokens_witcher))]

In [ ]:
# text_lotr, list_of_tokens_lotr = prep(path_to_all, lotr_order, max_len_of_sent=200, max_len_of_paragraph=500)
# print(len(text_lotr))
# text_lotr[np.argmin(np.array(list_of_tokens_lotr))], text_lotr[np.argmax(np.array(list_of_tokens_lotr))]

In [ ]:
text_all, list_of_tokens_all = prep(
    path_to_all,
    itertools.chain(witcher_order, lotr_order, got_order),
    max_len_of_sent=200,
    max_len_of_paragraph=161
)
print(len(text_all))
text_all[np.argmin(np.array(list_of_tokens_all))], text_all[np.argmax(np.array(list_of_tokens_all))]

In [ ]:
def get_last_state_dict(path_to_data):
    list_of_files = glob.glob(f"{path_to_data}/*")
    array = []
    for file in list_of_files:
        array.append((file, int(re.findall('\d+', file.split('/')[-1])[0])))
    array.sort(key= lambda x: x[1], reverse=True)
    path, epoch = array[0]
    print(f'last state dict in {path}. epoch {epoch}')
    return path, epoch

In [ ]:
# https://githubhelp.com/EmilyNLP/Fine-Tune-GPT2-to-Generate-Stories

# http://education.abcom.com/using-gpt-2-to-write-like-shakespeare/

Датасеты и даталоадеры

In [ ]:
class GameOfThronesDataset(Dataset):
    def __init__(self, list_of_sentences):
        self.labels=[]
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.data = self.tokenizer(list_of_sentences, padding=True, return_attention_mask=True)
        
        labels=[]
        for ids, attention_mask in zip(self.data['input_ids'], self.data['attention_mask']):
            label=ids.copy()
            real_len=sum(attention_mask)
            padding_len=len(attention_mask)-sum(attention_mask)
            label[:]=label[:real_len]+[-100]*padding_len
            labels.append(label)
            
        self.data['labels']=labels
        
        print('length of sent: ', len(self.data['input_ids'][0]))
        print('length of data: ', len(self.data['input_ids']))
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, index):

        return [torch.tensor(self.data['input_ids'][index], dtype=torch.long),
                torch.tensor(self.data['attention_mask'][index], dtype=torch.long),
                torch.tensor(self.data['labels'][index], dtype=torch.long)]

In [ ]:
text_dataset = GameOfThronesDataset(text_all)
Dataloader = DataLoader(text_dataset, batch_size=8)

In [ ]:
# train = text[:12000]
# test = text[12000:]
# train_dataset = GameOfThronesDataset2(train)
# test_dataset = GameOfThronesDataset2(test)
# train_dataloader = DataLoader(train_dataset, batch_size=8)
# test_dataloader = DataLoader(test_dataset, batch_size=8)

инициализация

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

num_train_epochs = 10
training_steps_per_epoch=len(Dataloader)
total_num_training_steps = int(training_steps_per_epoch * num_train_epochs)
weight_decay = 0
learning_rate = 5e-5
adam_epsilon = 1e-8
warmup = 0.1
warmup_steps = int(total_num_training_steps * warmup)

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_num_training_steps
)

# gc.collect()
# torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# path, epoch = get_last_state_dict(path_to_models)

In [ ]:
# model.load_state_dict(torch.load(path))
# model.to(device)

Обучение

In [ ]:
from tqdm import tqdm

def train(traindataloader, model, optimizer, scheduler, device, epochs, output_dir, output_prefix, save_model_on_epoch, validate_every, testdataloader=None):
    
    losses_train_overall = []
    losses_test_overall = []
    
    for i in range(epochs):
        
        print(f"Training epoch {i}")
        model.train()
        train_loss = []
        test_loss = []
        model.zero_grad() ###
        optimizer.zero_grad()
        for batch in tqdm(traindataloader, desc="train"):
            inputs, attention, labels = batch
            inputs = inputs.to(device)
#             inputs = inputs.squeeze(1).to(device)
            attention = attention.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=inputs, attention_mask=attention, labels=labels)
            loss = outputs[0]

            batch_loss = loss.cpu().item()
            train_loss.append(batch_loss)

            loss.backward()
            optimizer.step()
            scheduler.step()
            model.zero_grad() ###
            optimizer.zero_grad()
            del loss
            
        train_loss_mean = np.mean(train_loss)
        print(f'train loss = {train_loss_mean}')
        print()

#         if (i % validate_every == 0):
            
#             model.eval()
#             optimizer.zero_grad()
#             model.zero_grad()
            
#             with torch.no_grad():
                               
#                 for batch in tqdm(testdataloader, desc="eval"):
#                     inputs, attention, labels = batch
#                     inputs = inputs.to(device)
# #                     inputs = inputs.squeeze(1).to(device)
#                     attention = attention.to(device)
#                     labels = labels.to(device)
                    
#                     outputs = model(input_ids=inputs, attention_mask=attention, labels=labels)
#                     loss = outputs[0]  
                    
#                     batch_val_loss = loss.cpu().item()
#                     test_loss.append(batch_val_loss)
#                     del loss
                    
#                 test_loss_mean = np.mean(test_loss)
#                 print(f'validate loss = {test_loss_mean}')
#                 print()

        if save_model_on_epoch:
            torch.save(model.state_dict(), os.path.join(output_dir, f"{output_prefix}-{i}.pt"))

        losses_train_overall.append(train_loss_mean)
#         losses_test_overall.append(test_loss_mean)
    return losses_train_overall, losses_test_overall

list_train_loss2, list_test_loss2 =  train(
    Dataloader, 
    model, 
    optimizer, 
    scheduler, 
    device, 
    epochs=num_train_epochs, 
    output_dir="/kaggle/working", 
    output_prefix=output_prefix, 
    save_model_on_epoch=True, 
    validate_every=1,
    testdataloader=None
)

Кривые обучения

In [ ]:
def learning_curves(train, val, txt='loss'):
    plt.figure(figsize=(8,6))
    plt.plot(range(1, len(train)+1), train, label='train {}'.format(txt))
    plt.plot(range(1, len(val)+1), val, label='validation {}'.format(txt))

    plt.title('Training history', fontsize=14)
    plt.ylabel('{}'.format(txt), fontsize=14)
    plt.xlabel('Epoch', fontsize=14)
    plt.legend(fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.grid()
    plt.show()

In [ ]:
learning_curves(list_train_loss2, list_test_loss2)

In [ ]:
# from itertools import chain
# learning_curves(list(a), list(b))

INFERENCE

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import glob
import re

path_to_models = '/kaggle/input/modelsnapshots'

# path_to_all = '/kaggle/input/fantasy-books'
# output_prefix = 'fantasy'

In [ ]:
def get_last_state_dict(path_to_data):
    list_of_files = glob.glob(f"{path_to_data}/*")
    array = []
    for file in list_of_files:
        array.append((file, int(re.findall('\d+', file.split('/')[-1])[0])))
    array.sort(key= lambda x: x[1], reverse=True)
    path, epoch = array[0]
    print(f'last state dict in {path}. epoch {epoch}')
    return path, epoch

In [ ]:
path, epoch = get_last_state_dict(path_to_models)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(torch.load(path))

Генерируем текст

In [ ]:
dataset_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
dataset_tokenizer.pad_token = dataset_tokenizer.eos_token


def generate(text, size='small'):
    """
    size = small, middle, big
    """
    dict_of_size = {'extrasmall':50, 'small': 100, 'middle': 150, 'big': 200}
    ids = dataset_tokenizer.encode(text, return_tensors='pt').to(device)
    greedy_output = model.generate(ids, do_sample=True, max_length=dict_of_size[size], top_k=40, top_p=0.95, num_return_sequences=3)
    result = dataset_tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    return result

generate(dataset_tokenizer.bos_token, 'middle')

In [ ]:
predict = generate(dataset_tokenizer.bos_token, 'extrasmall')
for _ in range(10):
    predict += generate(predict[-1], 'extrasmall')
    
predict